In [ ]:
import os
import re
import time
from bs4 import BeautifulSoup
import requests
import json
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import random
import pandas as pd

In [ ]:
### 辅助函数
# 获取当前已审核页面中的样点文本信息
def get_point_text_info(driver,data:list):
    """获取当前页面的样点文本信息"""
    # page_source 是您从selenium获取到的页面源代码
    page_source = driver.page_source
    # 使用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(page_source, 'html.parser')
    # 数据行都在一定的HTML结构内，例如在某个table或特定标签内
    for row in soup.find_all('tr'):  # 假设每行数据都在一个<tr>标签内
        # 通过选择class为ant-table-cell的<td>标签，并限制结果数量为17来获取前17个信息
        cols = row.find_all('td', class_='ant-table-cell', limit=17)
        if len(cols) == 17:  # 确保提取到了全部17个所需的列
            data_row = {
                '样点编号': cols[0].text.strip(),
                '坐落单位名称': cols[1].text.strip(),
                '样点类别': cols[2].text.strip(),
                '采样类型': cols[3].text.strip(),
                '土壤类型': cols[4].text.strip(), 
                '土地利用类型': cols[5].text.strip(),
                '是否水团样': cols[6].text.strip(),
                '是否盐碱地': cols[7].text.strip(),
                '布设位置经度': cols[8].text.strip(),
                '布设位置纬度': cols[9].text.strip(),
                '采样位置经度': cols[10].text.strip(),
                '采样位置纬度': cols[11].text.strip(),
                '是否修正': cols[12].text.strip(),
                '当前节点': cols[13].text.strip(),
                '调查时间': cols[14].text.strip(),
                '调查单位': cols[15].text.strip(),
                '调查人': cols[16].text.strip(),
                # '操作' 列被我们有意地忽略了，因为它是“详情”按钮
            }
            data.append(data_row)

# 获取当前未审核页面中的样点文本信息
def get_point_text_info_not(driver,data:list):
    """获取当前页面的样点文本信息"""
    # page_source 是您从selenium获取到的页面源代码
    page_source = driver.page_source
    # 使用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(page_source, 'html.parser')
    # 数据行都在一定的HTML结构内，例如在某个table或特定标签内
    for row in soup.find_all('tr'):  # 假设每行数据都在一个<tr>标签内
        # 通过选择class为ant-table-cell的<td>标签，并限制结果数量为17来获取前17个信息
        cols = row.find_all('td', class_='ant-table-cell', limit=18)
        if len(cols) == 18:  # 确保提取到了全部17个所需的列
            data_row = {
                '样点编号': cols[1].text.strip(),
                '坐落单位名称': cols[2].text.strip(),
                '样点类别': cols[3].text.strip(),
                '采样类型': cols[4].text.strip(),
                '土壤类型': cols[5].text.strip(), 
                '土地利用类型': cols[6].text.strip(),
                '是否水团样': cols[7].text.strip(),
                '是否盐碱地': cols[8].text.strip(),
                '布设位置经度': cols[9].text.strip(),
                '布设位置纬度': cols[10].text.strip(),
                '采样位置经度': cols[11].text.strip(),
                '采样位置纬度': cols[12].text.strip(),
                '是否修正': cols[13].text.strip(),
                '当前节点': cols[14].text.strip(),
                '调查时间': cols[15].text.strip(),
                '调查单位': cols[16].text.strip(),
                '调查人': cols[17].text.strip(),
                # '操作' 列被我们有意地忽略了，因为它是“详情”按钮
            }
            data.append(data_row)
# 获取待审核页面内容
def get_pending_approval_page_content(driver,data):
    """获取待审核页面表格内容"""
    # 针对待审核数据
    # 初始化翻页次数计数器
    page_count = 0
    while True:
        try:
            # 尝试查找“下一页”按钮，这里使用了CSS选择器，您也可以根据实际情况使用XPath或其他选择器
            next_page_button = driver.find_element(By.CSS_SELECTOR, "li.ant-pagination-next[title='下一页']")
            # 检查“下一页”按钮是否被禁用（即已经到达最后一页）
            # 这一步是可选的，取决于页面的实际行为
            if 'ant-pagination-disabled' in next_page_button.get_attribute('class'):
                print("已到达最后一页。")
                # 采集最后一页信息
                get_point_text_info_not(driver,data)
                break
            # 采集信息
            get_point_text_info_not(driver,data)
            # 点击“下一页”按钮
            next_page_button.click()
            # 增加翻页计数
            page_count += 1
        except NoSuchElementException:
            # 如果找不到“下一页”按钮，说明已经到达最后一页
            print("已经到达最后一页，无法继续翻页。")
            break
        except Exception as e:
            # 处理其他可能的异常
            print(f"在翻页过程中发生错误：{e}")
            break
        time.sleep(3)
    print(f"总共翻页次数：{page_count}")

# 获取已审核页面内容
def get_approved_page_content(driver,data):
    """获取已审核页面表格内容"""
    # 针对已审核数据
    # 初始化翻页次数计数器
    page_count = 0
    while True:
        try:
            # 尝试查找“下一页”按钮，这里使用了CSS选择器，您也可以根据实际情况使用XPath或其他选择器
            next_page_button = driver.find_element(By.CSS_SELECTOR, "li.ant-pagination-next[title='下一页']")
            # 检查“下一页”按钮是否被禁用（即已经到达最后一页）
            # 这一步是可选的，取决于页面的实际行为
            if 'ant-pagination-disabled' in next_page_button.get_attribute('class'):
                print("已到达最后一页。")
                # 采集最后一页信息
                get_point_text_info(driver,data)
                break
            # 采集信息
            get_point_text_info(driver,data)
            # 点击“下一页”按钮
            next_page_button.click()
            # 增加翻页计数
            page_count += 1
        except NoSuchElementException:
            # 如果找不到“下一页”按钮，说明已经到达最后一页
            print("已经到达最后一页，无法继续翻页。")
            break
        except Exception as e:
            # 处理其他可能的异常
            print(f"在翻页过程中发生错误：{e}")
            break
        time.sleep(3)
    print(f"总共翻页次数：{page_count}")



# 跳转到下一页
def step_next_page(driver):
    """跳转下一页"""
    # 使用类名和属性选择器
    next_page_button = driver.find_element(By.CSS_SELECTOR, "li.ant-pagination-next[title='下一页']")
    next_page_button.click()
    # # 使用XPath
    # next_page_button = driver.find_element(By.XPATH, "//li[@class='ant-pagination-next' and @title='下一页']")
    # next_page_button.click()
    # # 使用类名
    # next_page_button = driver.find_element(By.CLASS_NAME, "ant-pagination-next")
    # next_page_button.click()




In [ ]:
# 配置浏览器
# chrome.exe --remote-debugging-port=9999 --user-data-dir="D:\Selenium\AutomationProfile"
# chrome.exe --remote-debugging-port=9999 --user-data-dir="D:\Program Files\ChromeDir"
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "localhost:9999")
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(20)

In [ ]:
# 初始化数据收集容器
data = []

In [ ]:
# 获取待审核数据
get_pending_approval_page_content(driver,data)

In [ ]:
# 获取已审核数据
get_approved_page_content(driver,data)

In [ ]:
df = pd.DataFrame(data)
df.size/17
# 去除重复数据
df = df.drop_duplicates()
df.size/17

In [ ]:
df.head()

In [ ]:
# 采集地区
collection_name = "lstzmj"
root_path = r"F:\collection_spb"

In [ ]:
# 创建对应的文件夹
dir_path = os.path.join(root_path,collection_name)
# 检查路径是否存在，否则便创建
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
else:
    print("文件夹已存在")

In [ ]:
# 创建对应的表格信息保存文件夹
table_path = os.path.join(dir_path,"table_info")
# 检查路径是否存在，否则便创建
if not os.path.exists(table_path):
    os.makedirs(table_path)
else:
    print("文件夹已存在")

In [ ]:
df.to_excel(os.path.join(table_path,"table_info.xlsx"),index=False)

In [ ]:
# 定位包含详情按钮的所有<tr>元素
rows = driver.find_elements(By.CSS_SELECTOR, "tr.ant-table-row")

In [ ]:
rows

In [ ]:
# 遍历每个<tr>元素
for row in rows:
    # 在每个<tr>元素中定位详情按钮
    detail_button = row.find_element(By.CSS_SELECTOR, "button.ant-btn.ant-btn-link.ant-btn-lg")
    
    # 点击详情按钮
    detail_button.click()
    
    # 处理详情页内容，可能需要等待内容加载
    time.sleep(2)  # 根据实际情况调整等待时间

    # 定位返回按钮
    back_button = driver.find_element(By.CSS_SELECTOR, "button.ant-btn")

    # 点击返回按钮
    back_button.click()


In [ ]:
image_elements = driver.find_elements(By.TAG_NAME, "img")

In [ ]:
image_elements

In [ ]:
# 提取所有图片的src属性，即它们的链接
image_urls = [image.get_attribute('src') for image in image_elements]

# 输出图片链接
for url in image_urls:
    print(url)

In [ ]:
images_info = driver.execute_script("""
    var images = [];
    var imageElements = document.querySelectorAll('div.card-box div.ant-image img');
    imageElements.forEach(function(img) {
        var labelElement = img.nextElementSibling;
        var label = labelElement ? labelElement.innerText : '';
        images.push({'url': img.src, 'label': label});
    });
    return images;
""")
for info in images_info:
    print(info)


In [ ]:
import re
html_source = driver.page_source
pattern = re.compile(r'<div class="ant-image"[^>]*><img[^>]+src="([^"]+)"[^>]*>.*?<div class="img-text"[^>]*>([^<]+)</div>', re.DOTALL)
matches = pattern.findall(html_source)
for match in matches:
    print({'url': match[0], 'label': match[1]})


In [ ]:

# 获取“下一张”按钮
next_button = driver.find_element(By.CSS_SELECTOR, 'div.swiper-button-next')

images_info = []
num_images = len(driver.find_elements(By.CSS_SELECTOR, 'div.card-box div.ant-image img'))

# 遍历图片集合
for _ in range(num_images):
    # 等待图片和标签信息更新
    time.sleep(1)  # 根据实际情况调整等待时间

    # 获取当前显示的图片URL和标签信息
    img = driver.find_element(By.CSS_SELECTOR, 'div.card-box div.ant-image img')
    labelDiv = driver.find_element(By.CSS_SELECTOR, "div.green-area")
    labelType = labelDiv.find_element(By.CSS_SELECTOR, "div.photo > div:first-child").text.replace('照片类型: ', '').strip()
    labelAngle = labelDiv.find_element(By.CSS_SELECTOR, "div.photo > div:last-child").text.replace('拍摄角度: ', '').strip()
    labelTime = labelDiv.find_element(By.CSS_SELECTOR, ".time-area").text.replace('拍摄时间: ', '').strip()

    images_info.append({
        'url': img.get_attribute('src'),
        'label': {'type': labelType, 'angle': labelAngle, 'time': labelTime}
    })

    # 点击“下一张”按钮
    try:
        next_button.click()
    except:
        pass

# # 输出获取到的图片信息
# for info in images_info:
#     print(info)

In [ ]:

# 等待并点击下拉选择的触发器以展开选项列表
dropdown_trigger = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.ant-select-selector'))
)
dropdown_trigger.click()

# 增加一些等待时间，确保下拉列表的选项加载完成

# 等待并定位到特定的选项（例如“混合点照片”）
option_to_select = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//div[contains(@class, 'ant-select-item-option-content') and contains(text(), '混样点照片')]"))
)
option_to_select.click()


In [ ]:

# 定位所有包含文本的元素
text_elements = driver.find_elements(By.XPATH, '//*[text()]')

# # 提取并打印每个元素的文本
# for element in text_elements:
#     print(element.text)